# 1. Logistic Regression

In [37]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
# from config import db_password
from sqlalchemy import inspect
import matplotlib.pyplot as plt
import pandas as pd

business_df = pd.read_csv("../../Data/merged_datasets.csv")

# Categorizing restaurants based on stars ratings
business_df["Category"] = pd.cut(business_df["Stars_Rating"],bins=[0.9,3,5],
                                 labels=["Lower Performance", "Higher Performance"])


business_df["Median_Income(dollars)"] = pd.to_numeric(business_df["Median_Income(dollars)"], errors='coerce')
business_df["Mean_Income(dollars)"] = pd.to_numeric(business_df["Mean_Income(dollars)"], errors='coerce')

business_df = business_df.dropna(subset=['Median_Income(dollars)'])

In [38]:
business_df

,Unnamed: 0,Restaurant_ID,Restaurants_Name,Address,City,State,Postal_Code,Latitude,Longitude,Stars_Rating,...,"Percentage_EH_$35,000-$49,999","Percentage_EH_$50,000-$74,999","Percentage_EH_$75,000-$99,999","Percentage_EH_$100,000-$149,999","Percentage_EH_$150,000-$199,999","Percentage_EH_$200,000<",Median_Income(dollars),Mean_Income(dollars),Population,Category
0,0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,...,11.0,16.3,10.3,13.0,6.6,11.1,60676.0,91522,27034,Higher Performance
1,1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,...,19.6,23.0,13.0,7.8,2.0,0.9,47114.0,55452,34174,Higher Performance
2,2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,...,4.5,15.9,11.3,16.9,15.2,24.7,116368.0,162885,24172,Higher Performance
3,3,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,...,9.1,13.6,12.0,16.2,9.7,17.9,86147.0,128426,57253,Higher Performance
4,4,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,...,17.4,18.8,11.7,11.3,3.2,3.4,48782.0,65655,94481,Lower Performance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24753,24753,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,Philadelphia,PA,19128,40.032483,-75.214430,3.0,...,8.8,19.6,13.8,19.5,10.5,7.6,76854.0,95665,76129,Lower Performance
24754,24754,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,"166 2nd Ave N, Ste A",Nashville,TN,37201,36.163875,-86.776311,4.0,...,4.5,13.1,29.2,9.0,4.7,18.1,92827.0,120146,1819,Higher Performance
24755,24755,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,83702,43.616590,-116.202383,4.5,...,8.0,18.1,11.4,11.6,8.5,14.3,68165.0,114580,23263,Higher Performance
24756,24756,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,...,16.3,24.2,13.1,12.1,2.2,3.2,55688.0,67108,47974,Higher Performance


In [39]:
business_df["Category"].value_counts()

Higher Performance    18913
Lower Performance      5836
Name: Category, dtype: int64

In [3]:
business_df["Stars_Rating"].describe()

count    24749.000000
mean         3.665845
std          0.694539
min          1.000000
25%          3.500000
50%          4.000000
75%          4.000000
max          5.000000
Name: Stars_Rating, dtype: float64

In [4]:
business_df.columns

Index(['Unnamed: 0', 'Restaurant_ID', 'Restaurants_Name', 'Address', 'City',
       'State', 'Postal_Code', 'Latitude', 'Longitude', 'Stars_Rating',
       'Review_Count', 'Restaurants_Delivery', 'Outdoor_Seating',
       'Accepts_CreditCards', 'Price_Range', 'Alcohol', 'Good_For_Kids',
       'Reservations', 'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions', 'Total_Estimate_Households_per_Zip',
       'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households', 'Percentage_EH<$10,000',
       'Percentage_EH_$10,000-$14,999', 'Percentage_EH_$15,000-$24,999',
       'Percentage_EH_$25,000-$34,999', 'Percentage_EH_$35,000-$49,999',
       'Percentage_EH_$50,000-$74,999', 'Percentage_EH_$75,000-$99,999',
       'Percentage_EH_$100,000-$149,999', 'Percentage_EH_$150,000-$199,999',
       'Percentage_EH_$200,000<', 'Median_Income(dollars)',
       'Mean_Income(dolla

In [5]:
# Define features set
X = business_df[['Review_Count', 'Restaurants_Delivery', 'Outdoor_Seating',
       'Accepts_CreditCards', 'Price_Range', 'Alcohol', 'Good_For_Kids',
       'Reservations', 'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions',
                'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households',
                'Median_Income(dollars)', 'Population']]

In [6]:
# Define the target
y = business_df["Category"]
y

0        Higher Performance
1        Higher Performance
2        Higher Performance
3        Higher Performance
4         Lower Performance
                ...        
24753     Lower Performance
24754    Higher Performance
24755    Higher Performance
24756    Higher Performance
24757    Higher Performance
Name: Category, Length: 24749, dtype: category
Categories (2, object): ['Lower Performance' < 'Higher Performance']

In [7]:
# Split the model into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y, 
                                                   random_state=1, 
                                                    stratify=y)

In [8]:
# Instantiate a Logistic Regression Model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs", random_state=1)
model

LogisticRegression(random_state=1)

In [9]:
# Train the model
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [10]:
# Make predictions
y_pred = model.predict(X_test)

In [11]:
# Evaluate the Model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7642210730446024

In [12]:
# Calculate the confusion matrix 
from sklearn.metrics import confusion_matrix, classification_report

matrix=confusion_matrix(y_test, y_pred)
print(matrix)

[[   0 1205    0    0]
 [   0 3507    0    0]
 [   0  254    0    0]
 [   0 1222    0    0]]


In [13]:
report=classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

     Average       0.00      0.00      0.00      1205
        Good       0.57      1.00      0.72      3507
        Poor       0.00      0.00      0.00       254
  Successful       0.00      0.00      0.00      1222

    accuracy                           0.57      6188
   macro avg       0.14      0.25      0.18      6188
weighted avg       0.32      0.57      0.41      6188



/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

# Logistic Regression with Oversampling

In [12]:
from collections import Counter
Counter(y)

Counter({'Higher Performance': 18913, 'Lower Performance': 5836})

It seems like the classes are unbalanced, which might bias the results toward the majority classes. 

# Naive Random Oversampling


In random oversampling, instances of the minority class are randomly selected and added to the training set until the majority and minority classes are balanced.  

In [15]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'Higher Performance': 14184, 'Lower Performance': 14184})

In [16]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6154329570174318

In [18]:
y_pred_train = model.predict(X_train)
balanced_accuracy_score(y_train, y_pred_train)

0.6146207225742006

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[2210, 2519],
       [ 345, 1114]])

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                          pre       rec       spe        f1       geo       iba       sup

Higher Performance       0.86      0.47      0.76      0.61      0.60      0.35      4729
 Lower Performance       0.31      0.76      0.47      0.44      0.60      0.37      1459

       avg / total       0.73      0.54      0.69      0.57      0.60      0.35      6188



# SMOTE Oversampling

In SMOTE, like naive random oversampling, the size of the minority is increased. The key difference with random oversampling is how the minority class is increased in size. 

However, in SMOTE, new instances are interpolated. That is, for an instance from the minority class, new values are generated based on its distance from its neighbors.

**Random oversampling draws from existing observations, whereas SMOTE generates synthetic observations**.

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy="auto").fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled)

Counter({'Higher Performance': 14184, 'Lower Performance': 14184})

In [23]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.617285090420315

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[2234, 2495],
       [ 347, 1112]])

In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                          pre       rec       spe        f1       geo       iba       sup

Higher Performance       0.87      0.47      0.76      0.61      0.60      0.35      4729
 Lower Performance       0.31      0.76      0.47      0.44      0.60      0.37      1459

       avg / total       0.73      0.54      0.69      0.57      0.60      0.35      6188



# Cluster Centroids Undersampling

Akin to SMOTE, the algorithm identifies clusters of the majority class, then generates synthetic data points, called centroids, that are representative of the clusters. The majority class is then undersampled down to the size of the minority class. 

In [31]:
# Resample the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled)

Counter({'Higher Performance': 4377, 'Lower Performance': 4377})

In [32]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6025434622328708

In [35]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1751, 2978],
       [ 241, 1218]])

In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                          pre       rec       spe        f1       geo       iba       sup

Higher Performance       0.88      0.37      0.83      0.52      0.56      0.29      4729
 Lower Performance       0.29      0.83      0.37      0.43      0.56      0.32      1459

       avg / total       0.74      0.48      0.73      0.50      0.56      0.30      6188



# Combination of Over and Under-Sampling

A downside of oversampling with SMOTE is its reliance on the immediate neighbors of a data point. Because the algorithm doesn't see the overall distribution of data, the new data it creates can be heavily influenced by outliers. 

A downside of undersampling is that it involves loss of data and is not an option when the dataset is small. 

SMOTEEN combines the SMOTE and Edited Nearest Neighbors (ENN) algorithms. It includes the following steps: 
1. Oversample the minority class with SMOTE.
2. Clean the resulting data with an undersampling strategy. If the two of the nearest neighbors of a data point belong to different classes, the data point is dropped.

In [32]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Average': 3310, 'Good': 1184, 'Poor': 6738, 'Successful': 2781})

In [33]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.28698953796394144

In [35]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   3,    4, 1127,   71],
       [  31,   35, 2881,  560],
       [   0,    0,  251,    3],
       [   6,   13, 1023,  180]])

In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

    Average       0.07      0.00      0.99      0.00      0.05      0.00      1205
       Good       0.67      0.01      0.99      0.02      0.10      0.01      3507
       Poor       0.05      0.99      0.15      0.09      0.39      0.16       254
 Successful       0.22      0.15      0.87      0.18      0.36      0.12      1222

avg / total       0.44      0.08      0.93      0.05      0.15      0.04      6188

